# LME銅市場総合ダッシュボード

Bloomberg APIから取得したデータを統合して、LME銅市場の総合的な分析ダッシュボードを作成します。

## ダッシュボード内容
- 🔸 市場概況サマリー
- 📈 価格・スプレッド・在庫の統合ビュー
- 🌍 地域別・取引所別比較
- 📊 相関分析・リスク指標
- 🚨 アラート・異常値検知
- 📅 直近パフォーマンス

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")

# 自作モジュールのインポート
import sys
sys.path.append('../config')
from data_utils import DataFetcher
from database_config import VISUALIZATION_CONFIG

# DataFetcherの初期化
fetcher = DataFetcher()

print("🎯 LME銅市場総合ダッシュボード")
print("=" * 50)
print(f"📅 実行日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. データ統合取得

In [ ]:
# 全データの統合取得
print("📥 データ統合取得中...")

# 1. 価格データ
price_data = fetcher.get_copper_prices(days=365)
cash_prices = price_data[price_data['TenorTypeName'] == 'Cash'].copy()
print(f"✅ 価格データ: {len(price_data)} レコード")

# 2. テナーデータ
tenor_data = fetcher.get_tenor_spread_data(days=365)
print(f"✅ テナーデータ: {len(tenor_data)} レコード")

# 3. 在庫データ
lme_inventory = fetcher.get_lme_inventory(days=365)
other_inventory = fetcher.get_other_inventory(days=365)
print(f"✅ LME在庫データ: {len(lme_inventory)} レコード")
print(f"✅ 他取引所在庫データ: {len(other_inventory)} レコード")

# 4. 市場指標
market_indicators = fetcher.get_market_indicators(days=365)
print(f"✅ 市場指標データ: {len(market_indicators)} レコード")

# データ期間の確認
start_date = min([df['TradeDate'].min() if 'TradeDate' in df.columns 
                 else df['ReportDate'].min() for df in [price_data, lme_inventory, market_indicators]])
end_date = max([df['TradeDate'].max() if 'TradeDate' in df.columns 
               else df['ReportDate'].max() for df in [price_data, lme_inventory, market_indicators]])

print(f"\n📅 データ期間: {start_date} ～ {end_date}")
print(f"📊 分析期間: {(end_date - start_date).days} 日間")

## 2. データ前処理・統合

In [ ]:
# 価格データの整理
price_pivot = cash_prices.pivot_table(
    index='TradeDate',
    columns='ExchangeCode',
    values='LastPrice',
    aggfunc='first'
).fillna(method='ffill')

# LMEテナースプレッドの計算
lme_tenor = tenor_data[tenor_data['ExchangeCode'] == 'LME'].copy()
lme_prices_pivot = lme_tenor.pivot_table(
    index='TradeDate',
    columns='TenorTypeName',
    values='LastPrice',
    aggfunc='first'
).fillna(method='ffill')

# Cash/3M スプレッド
cash_3m_spread = None
if 'Cash' in lme_prices_pivot.columns and '3M Futures' in lme_prices_pivot.columns:
    cash_3m_spread = lme_prices_pivot['3M Futures'] - lme_prices_pivot['Cash']

# LME在庫合計
lme_inventory_total = lme_inventory.groupby('ReportDate')['TotalStock'].sum()

# 統合データフレームの作成
dashboard_data = pd.DataFrame()

if 'LME' in price_pivot.columns:
    dashboard_data['LME_Price'] = price_pivot['LME']
    
if cash_3m_spread is not None:
    dashboard_data['Cash_3M_Spread'] = cash_3m_spread
    
dashboard_data = dashboard_data.join(lme_inventory_total.rename('LME_Inventory'), how='outer')
dashboard_data = dashboard_data.fillna(method='ffill').dropna()

print(f"📊 統合データ: {len(dashboard_data)} 日間")
print(f"📈 利用可能な指標: {list(dashboard_data.columns)}")

# 最新値の確認
latest_data = dashboard_data.iloc[-1]
print("\n💰 最新値:")
for col, value in latest_data.items():
    if 'Price' in col:
        print(f"  {col}: {value:,.2f} USD/MT")
    elif 'Spread' in col:
        print(f"  {col}: {value:+.2f} USD/MT")
    elif 'Inventory' in col:
        print(f"  {col}: {value:,.0f} トン")
    else:
        print(f"  {col}: {value:,.2f}")

## 3. 市場概況サマリー

In [ ]:
# 市場概況の計算
summary_stats = {}

# 価格変動
if 'LME_Price' in dashboard_data.columns:
    lme_price = dashboard_data['LME_Price']
    
    summary_stats['current_price'] = lme_price.iloc[-1]
    summary_stats['price_change_1d'] = (lme_price.iloc[-1] - lme_price.iloc[-2]) / lme_price.iloc[-2] * 100 if len(lme_price) > 1 else 0
    summary_stats['price_change_7d'] = (lme_price.iloc[-1] - lme_price.iloc[-7]) / lme_price.iloc[-7] * 100 if len(lme_price) > 7 else 0
    summary_stats['price_change_30d'] = (lme_price.iloc[-1] - lme_price.iloc[-30]) / lme_price.iloc[-30] * 100 if len(lme_price) > 30 else 0
    summary_stats['price_volatility_30d'] = lme_price.tail(30).pct_change().std() * np.sqrt(252) * 100

# スプレッド
if 'Cash_3M_Spread' in dashboard_data.columns:
    spread = dashboard_data['Cash_3M_Spread']
    
    summary_stats['current_spread'] = spread.iloc[-1]
    summary_stats['spread_change_7d'] = spread.iloc[-1] - spread.iloc[-7] if len(spread) > 7 else 0
    summary_stats['market_structure'] = "コンタンゴ" if spread.iloc[-1] > 0 else "バックワーデーション"
    summary_stats['spread_percentile'] = (spread <= spread.iloc[-1]).mean() * 100

# 在庫
if 'LME_Inventory' in dashboard_data.columns:
    inventory = dashboard_data['LME_Inventory']
    
    summary_stats['current_inventory'] = inventory.iloc[-1]
    summary_stats['inventory_change_7d'] = (inventory.iloc[-1] - inventory.iloc[-7]) / inventory.iloc[-7] * 100 if len(inventory) > 7 else 0
    summary_stats['inventory_percentile'] = (inventory <= inventory.iloc[-1]).mean() * 100

# サマリー表示
print("📋 市場概況サマリー")
print("=" * 30)

if 'current_price' in summary_stats:
    print(f"💰 LME銅価格: {summary_stats['current_price']:,.0f} USD/MT")
    print(f"   1日変動: {summary_stats['price_change_1d']:+.2f}%")
    print(f"   7日変動: {summary_stats['price_change_7d']:+.2f}%")
    print(f"   30日変動: {summary_stats['price_change_30d']:+.2f}%")
    print(f"   30日ボラティリティ: {summary_stats['price_volatility_30d']:.1f}%")

if 'current_spread' in summary_stats:
    print(f"\n📐 Cash/3M スプレッド: {summary_stats['current_spread']:+.2f} USD/MT")
    print(f"   市場構造: {summary_stats['market_structure']}")
    print(f"   7日変動: {summary_stats['spread_change_7d']:+.2f} USD/MT")
    print(f"   歴史的パーセンタイル: {summary_stats['spread_percentile']:.1f}%")

if 'current_inventory' in summary_stats:
    print(f"\n📦 LME在庫: {summary_stats['current_inventory']:,.0f} トン")
    print(f"   7日変動: {summary_stats['inventory_change_7d']:+.2f}%")
    print(f"   歴史的パーセンタイル: {summary_stats['inventory_percentile']:.1f}%")

# アラート判定
alerts = []
if 'price_volatility_30d' in summary_stats and summary_stats['price_volatility_30d'] > 40:
    alerts.append("⚠️ 高ボラティリティ状態")
if 'current_spread' in summary_stats and abs(summary_stats['current_spread']) > 100:
    alerts.append("🚨 異常スプレッド水準")
if 'inventory_percentile' in summary_stats and summary_stats['inventory_percentile'] < 10:
    alerts.append("📉 在庫水準極めて低い")
elif 'inventory_percentile' in summary_stats and summary_stats['inventory_percentile'] > 90:
    alerts.append("📈 在庫水準極めて高い")

if alerts:
    print("\n🚨 アラート:")
    for alert in alerts:
        print(f"   {alert}")
else:
    print("\n✅ アラートなし")

## 4. メインダッシュボード

In [ ]:
# メインダッシュボードの作成
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=(
        'LME銅価格推移', 'Cash/3M スプレッド', 'LME在庫推移',
        '取引所別価格比較', 'スプレッド分布', '在庫・価格相関',
        '価格変動率', 'リスク指標', '市場構造推移'
    ),
    specs=[
        [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
        [{"secondary_y": False}, {"type": "histogram"}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]
    ],
    vertical_spacing=0.08,
    horizontal_spacing=0.08
)

# 1. LME銅価格推移
if 'LME_Price' in dashboard_data.columns:
    fig.add_trace(
        go.Scatter(
            x=dashboard_data.index,
            y=dashboard_data['LME_Price'],
            mode='lines',
            name='LME銅価格',
            line=dict(color='#1f77b4', width=2),
            hovertemplate='日付: %{x}<br>価格: %{y:,.0f} USD/MT<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 30日移動平均
    ma30 = dashboard_data['LME_Price'].rolling(window=30).mean()
    fig.add_trace(
        go.Scatter(
            x=dashboard_data.index,
            y=ma30,
            mode='lines',
            name='30日MA',
            line=dict(color='red', width=1, dash='dash'),
            hovertemplate='日付: %{x}<br>MA30: %{y:,.0f} USD/MT<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Cash/3M スプレッド
if 'Cash_3M_Spread' in dashboard_data.columns:
    spread_colors = ['red' if x < 0 else 'green' for x in dashboard_data['Cash_3M_Spread']]
    
    fig.add_trace(
        go.Scatter(
            x=dashboard_data.index,
            y=dashboard_data['Cash_3M_Spread'],
            mode='lines',
            name='Cash/3M スプレッド',
            line=dict(color='blue', width=2),
            fill='tonexty',
            hovertemplate='日付: %{x}<br>スプレッド: %{y:+.2f} USD/MT<extra></extra>'
        ),
        row=1, col=2
    )
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=1, col=2)

# 3. LME在庫推移
if 'LME_Inventory' in dashboard_data.columns:
    fig.add_trace(
        go.Scatter(
            x=dashboard_data.index,
            y=dashboard_data['LME_Inventory'],
            mode='lines',
            name='LME在庫',
            line=dict(color='orange', width=2),
            hovertemplate='日付: %{x}<br>在庫: %{y:,.0f} トン<extra></extra>'
        ),
        row=1, col=3
    )

# 4. 取引所別価格比較
colors_exchanges = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i, exchange in enumerate(price_pivot.columns):
    fig.add_trace(
        go.Scatter(
            x=price_pivot.index[-30:],  # 過去30日
            y=price_pivot[exchange].tail(30),
            mode='lines',
            name=f'{exchange}価格',
            line=dict(color=colors_exchanges[i % len(colors_exchanges)], width=2)
        ),
        row=2, col=1
    )

# 5. スプレッド分布
if 'Cash_3M_Spread' in dashboard_data.columns:
    fig.add_trace(
        go.Histogram(
            x=dashboard_data['Cash_3M_Spread'],
            nbinsx=30,
            name='スプレッド分布',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=2, col=2
    )

# 6. 在庫・価格相関
if 'LME_Price' in dashboard_data.columns and 'LME_Inventory' in dashboard_data.columns:
    fig.add_trace(
        go.Scatter(
            x=dashboard_data['LME_Inventory'],
            y=dashboard_data['LME_Price'],
            mode='markers',
            name='在庫 vs 価格',
            marker=dict(color='purple', size=4, opacity=0.6),
            hovertemplate='在庫: %{x:,.0f} トン<br>価格: %{y:,.0f} USD/MT<extra></extra>'
        ),
        row=2, col=3
    )

# 7. 価格変動率
if 'LME_Price' in dashboard_data.columns:
    returns = dashboard_data['LME_Price'].pct_change().dropna() * 100
    fig.add_trace(
        go.Scatter(
            x=returns.index,
            y=returns,
            mode='lines',
            name='日次変動率',
            line=dict(color='red', width=1),
            hovertemplate='日付: %{x}<br>変動率: %{y:.2f}%<extra></extra>'
        ),
        row=3, col=1
    )
    
    fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=3, col=1)

# 8. ボラティリティ（リスク指標）
if 'LME_Price' in dashboard_data.columns:
    rolling_vol = dashboard_data['LME_Price'].pct_change().rolling(window=30).std() * np.sqrt(252) * 100
    fig.add_trace(
        go.Scatter(
            x=rolling_vol.index,
            y=rolling_vol,
            mode='lines',
            name='30日ボラティリティ',
            line=dict(color='green', width=2),
            hovertemplate='日付: %{x}<br>ボラティリティ: %{y:.1f}%<extra></extra>'
        ),
        row=3, col=2
    )

# 9. 市場構造推移（コンタンゴ・バックワーデーション）
if 'Cash_3M_Spread' in dashboard_data.columns:
    # 7日移動平均でスムーズ化
    spread_ma = dashboard_data['Cash_3M_Spread'].rolling(window=7).mean()
    market_structure = spread_ma.apply(lambda x: 1 if x > 0 else -1)  # 1=コンタンゴ, -1=バックワーデーション
    
    fig.add_trace(
        go.Scatter(
            x=market_structure.index,
            y=market_structure,
            mode='lines',
            name='市場構造',
            line=dict(color='navy', width=2),
            hovertemplate='日付: %{x}<br>構造: %{text}<extra></extra>',
            text=["コンタンゴ" if x > 0 else "バックワーデーション" for x in market_structure]
        ),
        row=3, col=3
    )

# レイアウト設定
fig.update_layout(
    height=900,
    title_text=f"🎯 LME銅市場総合ダッシュボード - {datetime.now().strftime('%Y-%m-%d')}",
    title_x=0.5,
    title_font_size=20,
    showlegend=False,
    font=dict(size=10)
)

# 各軸のタイトル設定
fig.update_xaxes(title_text="日付", row=1, col=1)
fig.update_xaxes(title_text="日付", row=1, col=2)
fig.update_xaxes(title_text="日付", row=1, col=3)
fig.update_xaxes(title_text="日付", row=2, col=1)
fig.update_xaxes(title_text="スプレッド (USD/MT)", row=2, col=2)
fig.update_xaxes(title_text="在庫 (トン)", row=2, col=3)
fig.update_xaxes(title_text="日付", row=3, col=1)
fig.update_xaxes(title_text="日付", row=3, col=2)
fig.update_xaxes(title_text="日付", row=3, col=3)

fig.update_yaxes(title_text="価格 (USD/MT)", row=1, col=1)
fig.update_yaxes(title_text="スプレッド (USD/MT)", row=1, col=2)
fig.update_yaxes(title_text="在庫 (トン)", row=1, col=3)
fig.update_yaxes(title_text="価格 (USD/MT)", row=2, col=1)
fig.update_yaxes(title_text="頻度", row=2, col=2)
fig.update_yaxes(title_text="価格 (USD/MT)", row=2, col=3)
fig.update_yaxes(title_text="変動率 (%)", row=3, col=1)
fig.update_yaxes(title_text="ボラティリティ (%)", row=3, col=2)
fig.update_yaxes(title_text="市場構造", row=3, col=3)

fig.show()

## 5. 地域別・取引所別詳細分析

In [ ]:
# 地域別・取引所別分析ダッシュボード
fig_regional = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'LME地域別在庫構成', '取引所別価格比較（正規化）', 
        'LME地域別在庫推移', '取引所別価格相関'
    ),
    specs=[
        [{"type": "pie"}, {"secondary_y": False}],
        [{"secondary_y": False}, {"secondary_y": False}]
    ],
    vertical_spacing=0.1
)

# 1. LME地域別在庫構成（最新）
latest_regional = lme_inventory[lme_inventory['ReportDate'] == lme_inventory['ReportDate'].max()]
regional_composition = latest_regional.groupby('RegionCode')['TotalStock'].sum()

fig_regional.add_trace(
    go.Pie(
        labels=regional_composition.index,
        values=regional_composition.values,
        name="地域別在庫",
        hovertemplate='<b>%{label}</b><br>' +
                     '在庫: %{value:,.0f} トン<br>' +
                     '構成比: %{percent}<extra></extra>'
    ),
    row=1, col=1
)

# 2. 取引所別価格比較（正規化）
if len(price_pivot.columns) > 1:
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    normalized_prices = pd.DataFrame(
        scaler.fit_transform(price_pivot.dropna()),
        index=price_pivot.dropna().index,
        columns=price_pivot.columns
    )
    
    colors_norm = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    for i, exchange in enumerate(normalized_prices.columns):
        fig_regional.add_trace(
            go.Scatter(
                x=normalized_prices.index,
                y=normalized_prices[exchange],
                mode='lines',
                name=f'{exchange} (正規化)',
                line=dict(color=colors_norm[i % len(colors_norm)], width=2)
            ),
            row=1, col=2
        )

# 3. LME地域別在庫推移
regional_pivot = lme_inventory.pivot_table(
    index='ReportDate',
    columns='RegionCode', 
    values='TotalStock',
    aggfunc='first'
).fillna(method='ffill')

colors_regions = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']
for i, region in enumerate(regional_pivot.columns):
    fig_regional.add_trace(
        go.Scatter(
            x=regional_pivot.index,
            y=regional_pivot[region],
            mode='lines',
            name=f'{region} 在庫',
            line=dict(color=colors_regions[i % len(colors_regions)], width=2),
            hovertemplate='日付: %{x}<br>在庫: %{y:,.0f} トン<extra></extra>'
        ),
        row=2, col=1
    )

# 4. 取引所別価格相関（ヒートマップ）
if len(price_pivot.columns) > 1:
    price_corr = price_pivot.corr()
    
    # Plotlyでヒートマップ
    fig_regional.add_trace(
        go.Heatmap(
            z=price_corr.values,
            x=price_corr.columns,
            y=price_corr.columns,
            colorscale='RdBu',
            zmid=0,
            text=price_corr.round(3).values,
            texttemplate="%{text}",
            textfont={"size":10},
            hovertemplate='%{y} vs %{x}<br>相関: %{z:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

# レイアウト設定
fig_regional.update_layout(
    height=700,
    title_text="🌍 地域別・取引所別詳細分析",
    title_x=0.5,
    showlegend=True
)

fig_regional.update_xaxes(title_text="日付", row=1, col=2)
fig_regional.update_xaxes(title_text="日付", row=2, col=1)
fig_regional.update_xaxes(title_text="取引所", row=2, col=2)

fig_regional.update_yaxes(title_text="正規化価格 (0-1)", row=1, col=2)
fig_regional.update_yaxes(title_text="在庫 (トン)", row=2, col=1)
fig_regional.update_yaxes(title_text="取引所", row=2, col=2)

fig_regional.show()

# 地域別統計サマリー
print("🌍 地域別在庫統計:")
print(regional_composition.to_string())
print(f"\n🏢 取引所別価格相関:")
if len(price_pivot.columns) > 1:
    print(price_corr.to_string())

## 6. リスク・アラート分析

In [ ]:
# リスク指標の計算
risk_metrics = {}

if 'LME_Price' in dashboard_data.columns:
    lme_price = dashboard_data['LME_Price']
    returns = lme_price.pct_change().dropna()
    
    # VaR計算（5%信頼水準）
    var_5 = np.percentile(returns, 5) * 100
    risk_metrics['VaR_5%'] = var_5
    
    # 最大ドローダウン
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max * 100
    max_drawdown = drawdown.min()
    risk_metrics['Max_Drawdown'] = max_drawdown
    
    # シャープレシオ（リスクフリーレート0%仮定）
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252)
    risk_metrics['Sharpe_Ratio'] = sharpe_ratio
    
    # 連続上昇・下落日数
    price_changes = lme_price.diff()
    up_days = 0
    down_days = 0
    current_streak = 0
    
    for change in price_changes.dropna().tail(10):
        if change > 0:
            if current_streak >= 0:
                current_streak += 1
            else:
                current_streak = 1
        elif change < 0:
            if current_streak <= 0:
                current_streak -= 1
            else:
                current_streak = -1
    
    risk_metrics['Current_Streak'] = current_streak

# 異常値検知
anomalies = []

if 'LME_Price' in dashboard_data.columns:
    # 価格の3σ異常値
    price_z_score = (lme_price - lme_price.mean()) / lme_price.std()
    if abs(price_z_score.iloc[-1]) > 3:
        anomalies.append(f"価格異常値 (Z-score: {price_z_score.iloc[-1]:.2f})")

if 'Cash_3M_Spread' in dashboard_data.columns:
    # スプレッドの異常値
    spread = dashboard_data['Cash_3M_Spread']
    spread_z_score = (spread - spread.mean()) / spread.std()
    if abs(spread_z_score.iloc[-1]) > 2.5:
        anomalies.append(f"スプレッド異常値 (Z-score: {spread_z_score.iloc[-1]:.2f})")

if 'LME_Inventory' in dashboard_data.columns:
    # 在庫の急激な変化
    inventory = dashboard_data['LME_Inventory']
    inventory_change = inventory.pct_change().abs()
    if len(inventory_change) > 1 and inventory_change.iloc[-1] > 0.1:  # 10%以上の変化
        anomalies.append(f"在庫急変 ({inventory_change.iloc[-1]*100:.1f}%)")

# リスク分析の可視化
fig_risk = make_subplots(
    rows=2, cols=2,
    subplot_titles=('ドローダウン分析', 'リターン分布', 'ボラティリティ推移', 'リスク指標サマリー'),
    specs=[
        [{"secondary_y": False}, {"type": "histogram"}],
        [{"secondary_y": False}, {"type": "table"}]
    ],
    vertical_spacing=0.1
)

# 1. ドローダウン分析
if 'LME_Price' in dashboard_data.columns:
    fig_risk.add_trace(
        go.Scatter(
            x=drawdown.index,
            y=drawdown,
            mode='lines',
            name='ドローダウン',
            line=dict(color='red', width=2),
            fill='tonexty',
            hovertemplate='日付: %{x}<br>ドローダウン: %{y:.2f}%<extra></extra>'
        ),
        row=1, col=1
    )

# 2. リターン分布
if 'LME_Price' in dashboard_data.columns:
    fig_risk.add_trace(
        go.Histogram(
            x=returns * 100,
            nbinsx=30,
            name='リターン分布',
            marker_color='lightgreen',
            opacity=0.7
        ),
        row=1, col=2
    )

# 3. ボラティリティ推移
if 'LME_Price' in dashboard_data.columns:
    rolling_vol_risk = returns.rolling(window=30).std() * np.sqrt(252) * 100
    
    fig_risk.add_trace(
        go.Scatter(
            x=rolling_vol_risk.index,
            y=rolling_vol_risk,
            mode='lines',
            name='30日ボラティリティ',
            line=dict(color='purple', width=2),
            hovertemplate='日付: %{x}<br>ボラティリティ: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 高ボラティリティレベル（40%）
    fig_risk.add_hline(y=40, line_dash="dash", line_color="red", opacity=0.7, row=2, col=1)

# 4. リスク指標テーブル
if risk_metrics:
    risk_table_data = [
        ['指標', '値'],
        ['VaR (5%)', f'{risk_metrics.get("VaR_5%", 0):.2f}%'],
        ['最大ドローダウン', f'{risk_metrics.get("Max_Drawdown", 0):.2f}%'],
        ['シャープレシオ', f'{risk_metrics.get("Sharpe_Ratio", 0):.2f}'],
        ['現在の連続日数', f'{risk_metrics.get("Current_Streak", 0)}日']
    ]
    
    fig_risk.add_trace(
        go.Table(
            header=dict(values=risk_table_data[0], fill_color='lightblue'),
            cells=dict(values=list(zip(*risk_table_data[1:])), fill_color='white')
        ),
        row=2, col=2
    )

# レイアウト設定
fig_risk.update_layout(
    height=700,
    title_text="🚨 リスク・アラート分析",
    title_x=0.5,
    showlegend=False
)

fig_risk.update_xaxes(title_text="日付", row=1, col=1)
fig_risk.update_xaxes(title_text="日次リターン (%)", row=1, col=2)
fig_risk.update_xaxes(title_text="日付", row=2, col=1)

fig_risk.update_yaxes(title_text="ドローダウン (%)", row=1, col=1)
fig_risk.update_yaxes(title_text="頻度", row=1, col=2)
fig_risk.update_yaxes(title_text="ボラティリティ (%)", row=2, col=1)

fig_risk.show()

# リスク分析結果の表示
print("🚨 リスク分析結果")
print("=" * 30)

for metric, value in risk_metrics.items():
    if 'VaR' in metric:
        print(f"{metric}: {value:.2f}%")
    elif 'Drawdown' in metric:
        print(f"{metric}: {value:.2f}%")
    elif 'Sharpe' in metric:
        print(f"{metric}: {value:.2f}")
    elif 'Streak' in metric:
        direction = "上昇" if value > 0 else "下落" if value < 0 else "横ばい"
        print(f"{metric}: {abs(value)}日連続{direction}")
    else:
        print(f"{metric}: {value:.2f}")

if anomalies:
    print("\n⚠️ 検出された異常:")
    for anomaly in anomalies:
        print(f"  - {anomaly}")
else:
    print("\n✅ 異常値検出なし")

## 7. 総合評価・推奨アクション

In [ ]:
# 総合評価の生成
print("🎯 総合評価・推奨アクション")
print("=" * 40)

# スコアリング（0-100点）
scores = {}

# 価格トレンドスコア
if 'price_change_30d' in summary_stats:
    if summary_stats['price_change_30d'] > 10:
        scores['price_trend'] = 85  # 強い上昇トレンド
    elif summary_stats['price_change_30d'] > 5:
        scores['price_trend'] = 70  # 上昇トレンド
    elif summary_stats['price_change_30d'] > -5:
        scores['price_trend'] = 50  # 横ばい
    elif summary_stats['price_change_30d'] > -10:
        scores['price_trend'] = 30  # 下落トレンド
    else:
        scores['price_trend'] = 15  # 強い下落トレンド

# ボラティリティスコア（低い方が良い）
if 'price_volatility_30d' in summary_stats:
    vol = summary_stats['price_volatility_30d']
    if vol < 20:
        scores['volatility'] = 90
    elif vol < 30:
        scores['volatility'] = 70
    elif vol < 40:
        scores['volatility'] = 50
    elif vol < 60:
        scores['volatility'] = 30
    else:
        scores['volatility'] = 10

# 市場構造スコア
if 'current_spread' in summary_stats:
    spread = summary_stats['current_spread']
    if spread > 50:  # 強いコンタンゴ
        scores['market_structure'] = 20
    elif spread > 20:
        scores['market_structure'] = 40
    elif spread > -20:
        scores['market_structure'] = 70  # 適正範囲
    elif spread > -50:
        scores['market_structure'] = 40
    else:  # 強いバックワーデーション
        scores['market_structure'] = 20

# 在庫レベルスコア
if 'inventory_percentile' in summary_stats:
    percentile = summary_stats['inventory_percentile']
    if 20 <= percentile <= 80:  # 適正範囲
        scores['inventory_level'] = 80
    elif 10 <= percentile <= 90:
        scores['inventory_level'] = 60
    else:  # 極端な水準
        scores['inventory_level'] = 30

# 総合スコア
if scores:
    overall_score = sum(scores.values()) / len(scores)
    scores['overall'] = overall_score

# スコア表示
print("📊 市場健全性スコア (0-100点):")
for category, score in scores.items():
    if category == 'overall':
        print(f"\n🎯 総合スコア: {score:.0f}点")
    else:
        category_names = {
            'price_trend': '価格トレンド',
            'volatility': 'ボラティリティ',
            'market_structure': '市場構造',
            'inventory_level': '在庫レベル'
        }
        print(f"  {category_names.get(category, category)}: {score:.0f}点")

# 推奨アクション
print("\n💡 推奨アクション:")

recommendations = []

# 価格トレンドベースの推奨
if 'price_change_30d' in summary_stats:
    if summary_stats['price_change_30d'] > 15:
        recommendations.append("📈 強い上昇トレンド：利益確定を検討")
    elif summary_stats['price_change_30d'] < -15:
        recommendations.append("📉 強い下落トレンド：押し目買いの機会を検討")

# ボラティリティベースの推奨
if 'price_volatility_30d' in summary_stats:
    if summary_stats['price_volatility_30d'] > 50:
        recommendations.append("⚠️ 高ボラティリティ：リスク管理を強化")
    elif summary_stats['price_volatility_30d'] < 15:
        recommendations.append("😴 低ボラティリティ：ブレイクアウトに注意")

# スプレッドベースの推奨
if 'current_spread' in summary_stats and 'spread_percentile' in summary_stats:
    spread = summary_stats['current_spread']
    percentile = summary_stats['spread_percentile']
    
    if spread > 0 and percentile > 90:
        recommendations.append("🔄 コンタンゴ極値：スプレッド縮小に注意")
    elif spread < 0 and percentile < 10:
        recommendations.append("🔄 バックワーデーション極値：需給逼迫状態")

# 在庫ベースの推奨
if 'inventory_percentile' in summary_stats:
    inv_percentile = summary_stats['inventory_percentile']
    if inv_percentile < 10:
        recommendations.append("📦 在庫極低水準：供給不足リスクに注意")
    elif inv_percentile > 90:
        recommendations.append("📦 在庫極高水準：需要低迷の可能性")

# リスク指標ベースの推奨
if 'Max_Drawdown' in risk_metrics:
    if risk_metrics['Max_Drawdown'] < -20:
        recommendations.append("📉 大幅ドローダウン：ポジション見直しを検討")

if 'VaR_5%' in risk_metrics:
    if risk_metrics['VaR_5%'] < -5:
        recommendations.append("⚠️ 高VaR：下方リスクが大きい状態")

# 異常値ベースの推奨
if anomalies:
    recommendations.append("🚨 異常値検出：市場動向を慎重に監視")

# 総合スコアベースの推奨
if 'overall' in scores:
    if scores['overall'] >= 80:
        recommendations.append("✅ 良好な市場環境：積極的な取引を検討")
    elif scores['overall'] >= 60:
        recommendations.append("⚖️ 中立的な市場環境：慎重な取引を推奨")
    elif scores['overall'] >= 40:
        recommendations.append("⚠️ 注意が必要な市場環境：リスク管理を重視")
    else:
        recommendations.append("🚨 リスクの高い市場環境：保守的な戦略を推奨")

# デフォルト推奨
if not recommendations:
    recommendations.append("📊 現在の市場データに基づく継続的な監視を推奨")

# 推奨アクション表示
for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")

# 次回更新予定
next_update = datetime.now() + timedelta(days=1)
print(f"\n🔄 次回更新予定: {next_update.strftime('%Y-%m-%d %H:%M')}")

print("\n✅ 総合ダッシュボード分析完了")

## 8. エクスポート機能

In [ ]:
# 分析結果のエクスポート
import os

# 出力ディレクトリの作成
output_dir = '../outputs'
os.makedirs(output_dir, exist_ok=True)

# 1. サマリーレポートの生成
report_date = datetime.now().strftime('%Y%m%d')
report_content = f"""
LME銅市場分析レポート
====================
作成日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
分析期間: {start_date} ～ {end_date}

【市場概況サマリー】
"""

for key, value in summary_stats.items():
    if 'price' in key.lower():
        report_content += f"{key}: {value:,.2f}\n"
    else:
        report_content += f"{key}: {value:.2f}\n"

report_content += "\n【リスク指標】\n"
for key, value in risk_metrics.items():
    report_content += f"{key}: {value:.2f}\n"

report_content += "\n【検出された異常値】\n"
if anomalies:
    for anomaly in anomalies:
        report_content += f"- {anomaly}\n"
else:
    report_content += "なし\n"

report_content += "\n【推奨アクション】\n"
for i, rec in enumerate(recommendations, 1):
    report_content += f"{i}. {rec}\n"

# レポートファイルの保存
report_filename = f'{output_dir}/copper_market_report_{report_date}.txt'
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(report_content)

print(f"📄 分析レポート保存: {report_filename}")

# 2. CSVデータの保存
if not dashboard_data.empty:
    csv_filename = f'{output_dir}/copper_dashboard_data_{report_date}.csv'
    dashboard_data.to_csv(csv_filename, encoding='utf-8')
    print(f"📊 ダッシュボードデータ保存: {csv_filename}")

# 3. 統計サマリーの保存
summary_df = pd.DataFrame({
    'Metric': list(summary_stats.keys()) + list(risk_metrics.keys()),
    'Value': list(summary_stats.values()) + list(risk_metrics.values())
})

summary_filename = f'{output_dir}/market_summary_{report_date}.csv'
summary_df.to_csv(summary_filename, index=False, encoding='utf-8')
print(f"📈 統計サマリー保存: {summary_filename}")

print("\n✅ エクスポート完了")
print(f"📁 出力フォルダ: {os.path.abspath(output_dir)}")

## 9. 実行結果サマリー

In [ ]:
# 最終サマリーの表示
print("🎯 LME銅市場総合ダッシュボード - 実行結果")
print("=" * 50)
print(f"📅 実行日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 分析期間: {(end_date - start_date).days} 日間 ({start_date} ～ {end_date})")

print("\n📈 取得データ:")
print(f"  - 価格データ: {len(price_data):,} レコード")
print(f"  - テナーデータ: {len(tenor_data):,} レコード")
print(f"  - LME在庫データ: {len(lme_inventory):,} レコード")
print(f"  - 他取引所在庫: {len(other_inventory):,} レコード")
print(f"  - 市場指標: {len(market_indicators):,} レコード")

print("\n🎯 分析内容:")
print("  ✅ 市場概況サマリー")
print("  ✅ 価格・スプレッド・在庫統合分析")
print("  ✅ 地域別・取引所別比較")
print("  ✅ リスク指標・異常値検知")
print("  ✅ 総合評価・推奨アクション")

if 'overall' in scores:
    print(f"\n🎯 総合評価: {scores['overall']:.0f}点/100点")
    
    if scores['overall'] >= 80:
        evaluation = "優良"
        emoji = "🟢"
    elif scores['overall'] >= 60:
        evaluation = "良好"
        emoji = "🟡"
    elif scores['overall'] >= 40:
        evaluation = "注意"
        emoji = "🟠"
    else:
        evaluation = "警戒"
        emoji = "🔴"
    
    print(f"📊 市場状況: {emoji} {evaluation}")

if alerts:
    print(f"\n🚨 アクティブアラート: {len(alerts)}件")
    for alert in alerts:
        print(f"  - {alert}")
else:
    print("\n✅ アクティブアラート: なし")

print(f"\n📁 出力ファイル:")
print(f"  - 分析レポート: copper_market_report_{report_date}.txt")
print(f"  - ダッシュボードデータ: copper_dashboard_data_{report_date}.csv")
print(f"  - 統計サマリー: market_summary_{report_date}.csv")

print("\n🔄 推奨監視頻度:")
if 'overall' in scores and scores['overall'] < 40:
    print("  ⚠️ 高頻度監視推奨（1日2-3回）")
elif 'price_volatility_30d' in summary_stats and summary_stats['price_volatility_30d'] > 40:
    print("  📊 頻繁監視推奨（1日1-2回）")
else:
    print("  📅 定期監視（1日1回）")

print("\n✅ ダッシュボード分析完了")
print("━" * 50)